In [9]:
include("./supplement.jl")
using Plots
using Distributed

In [ ]:
unitcell = Lattice([0, 0]; vectors=[[1, 0],[0, 1]])
cluster = Lattice(unitcell,(2,2),('p','p'))
hilbert = Hilbert(site=>Fock{:f}(1, 2) for site=1:length(cluster))
bs = Sector(hilbert, SpinfulParticle(4, 0.0))
t = Hopping(:t, Complex(-1.0), 1)
U = Hubbard(:U, Complex(0.0))
μ = Onsite(:μ, Complex(-0.0))
origiterms = (t, U, μ)
t_r = Hopping(:t, Complex(-1.0), 1)
af = Onsite(:af, Complex(0.0), MatrixCoupling(:, FID, :, σ"z", :); amplitude=antiferro([π, π]))
referterms = (t_r, U, μ, af)
neighbors = Neighbors(0=>0.0, 1=>1.0)
varparams = [(U = u, af = a) for u in [2,4,8,12,16], a in range(1e-9, 0.3, 25)]
rz = ReciprocalZone(reciprocals(cluster.vectors); length=100)
spawn(4)
@everywhere begin
    include("./supplement.jl")
end 
@time vcas = pmap(param -> VCA(:N, unitcell, cluster, hilbert, origiterms, referterms, bs, param; neighbors=neighbors, m=200), varparams)
@time gps = pmap(vca -> GrandPotential(:f, vca, rz, real(Parameters(vca.refergenerator)[:U]/2)), vcas)

f = plot(range(0, 0.3, 25), gps[1,:].-gps[1,1], label="U=2", legend=:topright, title="Ω-Ω₀ vs M")
plot!(range(0, 0.3, 25), gps[2,:].-gps[2,1], label="U=4", legend=:topright, title="Ω-Ω₀ vs M")
plot!(range(0, 0.3, 25), gps[3,:].-gps[3,1], label="U=8", legend=:topright, title="Ω-Ω₀ vs M")
plot!(range(0, 0.3, 25), gps[4,:].-gps[4,1], label="U=12", legend=:topright, title="Ω-Ω₀ vs M")
plot!(range(0, 0.3, 25), gps[5,:].-gps[5,1], label="U=16", legend=:topright, title="Ω-Ω₀ vs M")

In [ ]:
unitcell = Lattice([0, 0]; vectors=[[1, 0], [0, 1]])
cluster = Lattice(unitcell, (2,2), ('p','p'))
hilbert = Hilbert(site=>Fock{:f}(1, 2) for site=1:length(cluster))
bs = BinaryBases(1:4,5:8, 0.0)
t = Hopping(:t, -1.0, 1)
U = Hubbard(:U, 8.0)
μ = Onsite(:μ, -1.2)
coupling=Coupling(Index(:, FID(1, 1//2, 1)), Index(:, FID(1, -1//2, 1)))-Coupling(Index(:, FID(1, -1//2, 1)), Index(:, FID(1, 1//2, 1)))
origiterms = (t, U, μ)
neighbors = Neighbors(0=>0.0, 1=>1.0, 2=>√2)

s = Pairing(:s, 0.3, 0, coupling/2)
D = Pairing(:D, 0.3, 1, coupling/2; amplitude=phase_by_azimuth([(0,π,2π),(π/2,3π/2)],[1.0,-1.0]))
Es = Pairing(:Es, 0.3, 1, coupling/2)
Dxy = Pairing(:Dxy, 0.3, 2, coupling/2; amplitude=phase_by_azimuth([(π/4,5π/4),(3π/4,7π/4)],[1.0, -1.0]))

refertermss = [(t, U, μ, s),(t, U, μ, D),(t, U, μ, Es),(t, U, μ, Dxy)]
r = range(1e-9, 0.3, 25)
varparamss = [[(s = a,) for a in r], [(D = a,) for a in r], [(Es = a,) for a in r], [(Dxy = a,) for a in r]]
rz = ReciprocalZone(reciprocals(cluster.vectors); length=100)
spawn(4)
@everywhere begin
    include("./supplement.jl")
end 
@time vcas1 = pmap(param -> VCA(:A, unitcell, cluster, hilbert, origiterms, refertermss[1], bs, param; neighbors=neighbors, m=200), varparams1)
@time gps1 = pmap(vca -> GrandPotential(:f, vca, rz, 0.0), vcas1)
@time vcas2 = pmap(param -> VCA(:A, unitcell, cluster, hilbert, origiterms, refertermss[2], bs, param; neighbors=neighbors, m=200), varparams2)
@time gps2 = pmap(vca -> GrandPotential(:f, vca, rz, 0.0), vcas2)
@time vcas3 = pmap(param -> VCA(:A, unitcell, cluster, hilbert, origiterms, refertermss[3], bs, param; neighbors=neighbors, m=200), varparams3)
@time gps3 = pmap(vca -> GrandPotential(:f, vca, rz, 0.0), vcas3)
@time vcas4 = pmap(param -> VCA(:A, unitcell, cluster, hilbert, origiterms, refertermss[4], bs, param; neighbors=neighbors, m=200), varparams4)
@time gps4 = pmap(vca -> GrandPotential(:f, vca, rz, 0.0), vcas4)

f = plot(range(0, 0.3, 25), gps1, label="s", legend=:topright, title="Ω vs M")
plot!(range(0, 0.3, 25), gps2, label="d_{x²-y²}", legend=:topright, title="Ω vs M")
plot!(range(0, 0.3, 25), gps3, label="extended s", legend=:topright, title="Ω vs M")
plot!(range(0, 0.3, 25), gps4, label="d_{xy}", legend=:topright, title="Ω vs M")
